# 서울시 미용실 데이터 - NAVER Map(v4) 데이터 전처리(2)
## 작성자(최종수정일) : 서동원(2020.04.19)
- 검색 데이터 필요한 정보 추려내기
- (1) 중복 제거하기
- (2) 카테고리가 미용실 관련 데이터만 남기기
- (3) 주소가 서울인 데이터만 남기기

In [44]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import re
import requests
import time
pd.set_option('display.max_columns',50) # DataFrame truncation 없이 보기
pd.set_option('display.max_rows',100)
from tqdm import tqdm
import math

## 네이버 검색 데이터

In [45]:
# 파일 읽어오기
naver_search_raw = pd.read_csv('./data/naver_map_seoul_salon_output_data_2020-04-09_2020-04-19_modified.csv', index_col=0)
pre_count = len(naver_search_raw)

In [46]:
# 중복제거
naver_search_raw.drop_duplicates(inplace=True)
print('중복 제거 전 개수 :', pre_count)
print('중복 제거 후 개수 :', len(naver_search_raw))
pre_count = len(naver_search_raw)

중복 제거 전 개수 : 26808
중복 제거 후 개수 : 24121


In [47]:
naver_search_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24121 entries, 0 to 11
Data columns (total 9 columns):
업소ID       24121 non-null object
업소명        24119 non-null object
소재지전화번호    16865 non-null object
소재지도로명     24121 non-null object
카테고리       24121 non-null object
검색인덱스      24121 non-null int64
검색어        24121 non-null object
검색건수       24121 non-null int64
검색주소타입     12 non-null float64
dtypes: float64(1), int64(2), object(6)
memory usage: 1.8+ MB


In [48]:
# '미용' 단어가 들어간 카테고리만 뽑기
list(filter(lambda x: '미용' in x, naver_search_raw['카테고리'].unique()))

['생활,편의 > 미용실',
 '미용 > 미용실',
 '미용 > 머리염색',
 '미용 > 네일아트,네일샵',
 '미용 > 메이크업',
 '생활,편의 > 미용',
 '미용 > 피부,체형관리',
 '미용 > 마사지,지압',
 '미용 > 속눈썹증모,연장',
 '생활,편의 > 남성전문미용실',
 '미용 > 두피,탈모관리',
 '미용 > 경락마사지',
 '쇼핑,유통 > 미용기기,재료',
 '협회,단체 > 화장품,미용',
 '직업,기술학원 > 미용',
 '미용실 > 어린이미용실',
 '반려동물 > 반려동물미용',
 '미용 > 다이어트,비만',
 '직업,기술학원 > 애견미용']

### 미용이 들어간 카테고리 중 실제로 필요한 카테고리는 다음과 같다.
    - (1) '생활,편의 > 미용실'
    - (2) '미용 > 미용실'
    - (3) '미용 > 머리염색'
    - (4) '생활,편의 > 미용'
    - (5) '생활,편의 > 남성전문미용실'
    - (6) '미용실 > 어린이미용실'

In [49]:
# 필요한 카케고리 리스트로 저장
category_list = ['생활,편의 > 미용실'
                , '미용 > 미용실'
                , '미용 > 머리염색'
                , '생활,편의 > 미용'
                , '생활,편의 > 남성전문미용실'
                , '미용실 > 어린이미용실']
category_list

['생활,편의 > 미용실',
 '미용 > 미용실',
 '미용 > 머리염색',
 '생활,편의 > 미용',
 '생활,편의 > 남성전문미용실',
 '미용실 > 어린이미용실']

In [50]:
# 카테고리로 제거하기
naver_search_raw.drop(
      index=naver_search_raw[~naver_search_raw['카테고리'].isin(category_list)].index
    , inplace=True
)
print('카테고리로 제거 전 개수 :', pre_count)
print('카테고리로 제거 후 개수 :', len(naver_search_raw))
pre_count = len(naver_search_raw)

카테고리로 제거 전 개수 : 24121
카테고리로 제거 후 개수 : 21751


In [51]:
# 카테고리로 확인
# 남긴 카테고리 이외에는 검색되지 않는다.
naver_search_raw['카테고리'].unique()

array(['생활,편의 > 미용실', '미용 > 미용실', '미용 > 머리염색', '생활,편의 > 미용',
       '생활,편의 > 남성전문미용실', '미용실 > 어린이미용실'], dtype=object)

In [52]:
naver_search_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21751 entries, 0 to 11
Data columns (total 9 columns):
업소ID       21751 non-null object
업소명        21749 non-null object
소재지전화번호    15227 non-null object
소재지도로명     21751 non-null object
카테고리       21751 non-null object
검색인덱스      21751 non-null int64
검색어        21751 non-null object
검색건수       21751 non-null int64
검색주소타입     12 non-null float64
dtypes: float64(1), int64(2), object(6)
memory usage: 1.7+ MB


In [53]:
# 네이버 지도에 검색할떄 사용한 업소명인 공공데이터 자체에 중복데이터가 있을 수 있다.
# '업소ID', '업소명', '소재지전화번호', '소재지도로명', '카테고리'
# 5가지 컬럼으로 팬시색인하여 중복값을 다시 한번 제거한다.
naver_search_raw.drop(
      index = naver_search_raw[naver_search_raw[['업소ID', '업소명', '소재지전화번호', '소재지도로명', '카테고리']].duplicated()].index
    , inplace=True
)
print('팬시색인으로 중복 제거 전 개수 :', pre_count)
print('팬시색인으로 중복 제거 후 개수 :', len(naver_search_raw))
pre_count = len(naver_search_raw)

팬시색인으로 중복 제거 전 개수 : 21751
팬시색인으로 중복 제거 후 개수 : 14129


In [54]:
naver_search_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14129 entries, 12 to 26795
Data columns (total 9 columns):
업소ID       14129 non-null object
업소명        14128 non-null object
소재지전화번호    10070 non-null object
소재지도로명     14129 non-null object
카테고리       14129 non-null object
검색인덱스      14129 non-null int64
검색어        14129 non-null object
검색건수       14129 non-null int64
검색주소타입     0 non-null float64
dtypes: float64(1), int64(2), object(6)
memory usage: 1.1+ MB


In [55]:
# 업소명에 NaN이 들어있어 네이버에 업소ID로 검색해본 결과
# 네이버 플레이스에도 이름이 nan으로 처리되어있다. 네이버 자체의 실수인듯
# 확인 URL : https://store.naver.com/hairshops/detail?id=472920069
naver_search_raw[naver_search_raw['업소명'].isnull()]

,업소ID,업소명,소재지전화번호,소재지도로명,카테고리,검색인덱스,검색어,검색건수,검색주소타입
15848,s472920069,NaN,NaN,서울특별시 서대문구 연희로11길 30,"생활,편의 > 미용실",8640,헤어아트,32,NaN


In [56]:
# 주소에 '서울' 문자열이 포함되지 않은 데이터 제거
naver_search_raw = naver_search_raw[naver_search_raw['소재지도로명'].apply(lambda x: '서울' in x.split()[0])]
print('주소가 서울이 아닌 정보 제거 전 개수 :', pre_count)
print('주소가 서울이 아닌 정보 제거 후 개수 :', len(naver_search_raw))
pre_count = len(naver_search_raw)

주소가 서울이 아닌 정보 제거 전 개수 : 14129
주소가 서울이 아닌 정보 제거 후 개수 : 13398


In [57]:
naver_search_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13398 entries, 12 to 26795
Data columns (total 9 columns):
업소ID       13398 non-null object
업소명        13397 non-null object
소재지전화번호    9516 non-null object
소재지도로명     13398 non-null object
카테고리       13398 non-null object
검색인덱스      13398 non-null int64
검색어        13398 non-null object
검색건수       13398 non-null int64
검색주소타입     0 non-null float64
dtypes: float64(1), int64(2), object(6)
memory usage: 1.0+ MB


In [58]:
# 저장을 위해 필요한 컬럼만 컬럼명을 바꿔서 팬시색인한다.
naver_search_for_save = naver_search_raw[['업소ID', '업소명', '소재지전화번호', '소재지도로명', '카테고리']].rename(columns={
      '업소ID' : 'store_id'
    , '업소명' : 'store_name'
    , '소재지전화번호' : 'tel'
    , '소재지도로명' : 'addrRoad'
    , '카테고리' : 'category'
})

naver_search_for_save

,store_id,store_name,tel,addrRoad,category
12,s38544397,헤어벤쳐,02-548-2683,서울특별시 강남구 압구정로 113-22,"생활,편의 > 미용실"
13,s36435875,이철헤어커커 강남점,02-3473-2326,서울특별시 강남구 강남대로98길 9 의성빌딩 2층,"생활,편의 > 미용실"
14,s31525797,헤어컷코,02-529-8971,서울특별시 강남구 남부순환로359길 4,미용 > 머리염색
15,s18431392,헤어인러브,02-546-0366,서울특별시 강남구 선릉로148길 39 2층,"생활,편의 > 미용실"
17,s12005487,사자헤어,02-547-3684,서울특별시 강남구 압구정로46길 27 명원빌딩,"생활,편의 > 미용실"
...,...,...,...,...,...
26788,s1855668222,연우헤어,02-2208-0825,서울특별시 중랑구 봉우재로 155 1층 108호,"생활,편의 > 미용실"
26789,k1921641224,포미헤어,02-433-3379,서울특별시 중랑구 동일로144길 26,"생활,편의 > 미용실"
26791,s36317225,SALON DE 로이,02-977-5677,서울특별시 중랑구 동일로 942 1층,"생활,편의 > 미용실"
26792,s1197075765,에이데이즈 헤어살롱,02-6925-2378,서울특별시 중랑구 봉화산로 232 2층,"생활,편의 > 미용실"


In [59]:
# DF를 csv파일로 저장
naver_search_for_save.to_csv(
      './data/naver_map_seoul_salon_output_data_2020-04-09_2020-04-19_final.csv'
    , sep=','
    , encoding='utf-8'
)

In [43]:
# 데이터 확인
pd.read_csv('./data/naver_map_seoul_salon_output_data_2020-04-09_2020-04-19_final.csv', index_col=0)


,store_id,store_name,tel,addrRoad,category
12,s38544397,헤어벤쳐,02-548-2683,서울특별시 강남구 압구정로 113-22,"생활,편의 > 미용실"
13,s36435875,이철헤어커커 강남점,02-3473-2326,서울특별시 강남구 강남대로98길 9 의성빌딩 2층,"생활,편의 > 미용실"
14,s31525797,헤어컷코,02-529-8971,서울특별시 강남구 남부순환로359길 4,미용 > 머리염색
15,s18431392,헤어인러브,02-546-0366,서울특별시 강남구 선릉로148길 39 2층,"생활,편의 > 미용실"
17,s12005487,사자헤어,02-547-3684,서울특별시 강남구 압구정로46길 27 명원빌딩,"생활,편의 > 미용실"
...,...,...,...,...,...
26788,s1855668222,연우헤어,02-2208-0825,서울특별시 중랑구 봉우재로 155 1층 108호,"생활,편의 > 미용실"
26789,k1921641224,포미헤어,02-433-3379,서울특별시 중랑구 동일로144길 26,"생활,편의 > 미용실"
26791,s36317225,SALON DE 로이,02-977-5677,서울특별시 중랑구 동일로 942 1층,"생활,편의 > 미용실"
26792,s1197075765,에이데이즈 헤어살롱,02-6925-2378,서울특별시 중랑구 봉화산로 232 2층,"생활,편의 > 미용실"
